Este notebook contiene todo el pre-processing realizado al CSV de reviews de alaska.

**IMPORTAMOS LIBRERIAS**

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

**DESCARGAMOS EL CHECKPOINT/PESOS DEL MODELO A UTILIZAR**

In [3]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

**IMPORTAMOS EL DATASET**

In [4]:
data = load_dataset('csv', data_files='../data/alaska.csv', split='train')

Found cached dataset csv (C:/Users/camat/.cache/huggingface/datasets/csv/default-4b39212cdd12efcb/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


**TRANSFORMACIONES**

In [5]:
data = data.select_columns(['text', 'rating'])

In [6]:
data = data.filter(lambda x: x['text'] is not None)

Loading cached processed dataset at C:\Users\camat\.cache\huggingface\datasets\csv\default-4b39212cdd12efcb\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-fb58eabecf92ce4a.arrow


In [7]:
def lowercase_function(examples):
    return {'text': examples['text'].lower()}

In [8]:
data = data.map(lowercase_function)

Loading cached processed dataset at C:\Users\camat\.cache\huggingface\datasets\csv\default-4b39212cdd12efcb\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-69a40fb59d68f503.arrow


In [9]:
def compute_review_length(example):
    return {"review_length": len(example["text"].split())}

In [10]:
data = data.map(compute_review_length)

Loading cached processed dataset at C:\Users\camat\.cache\huggingface\datasets\csv\default-4b39212cdd12efcb\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-7536b474b01b669c.arrow


In [11]:
data.sort('review_length', reverse=True)['review_length'][:3]

Loading cached sorted indices for dataset at C:\Users\camat\.cache\huggingface\datasets\csv\default-4b39212cdd12efcb\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-1a0fabf1944782f4.arrow


[817, 777, 732]

In [12]:
data.sort('review_length', reverse=False)[:3]

Loading cached sorted indices for dataset at C:\Users\camat\.cache\huggingface\datasets\csv\default-4b39212cdd12efcb\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-815efd6e8d12318d.arrow


{'rating': [5, 5, 5],
 'text': ['georgeous', 'nice', 'great'],
 'review_length': [1, 1, 1]}

**TOKENIZAMOS USANDO TOKENIZER**

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [14]:
data = data.map(tokenize_function, batched=True, batch_size=1000)

In [67]:
data = data.map(lambda example: {'rating':example['rating'] - 1})

Loading cached processed dataset at C:\Users\camat\.cache\huggingface\datasets\csv\default-4b39212cdd12efcb\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-92ef839242081cf3.arrow


In [69]:
data = data.shuffle(seed=2023)

In [70]:
temp_split = data.train_test_split(test_size=0.1, seed=2023)['test']
temp_split = temp_split.train_test_split(test_size=0.8, seed=2023)
temp_split['validation'] = temp_split.pop('train')
temp_split['train'] = data

In [34]:
temp_split.save_to_disk('../test_data')